# Biostat M280 Homework 4

**Due June 12 @ 11:59PM**

In this homework, we build a classifer for handwritten digit recognition. Following figure shows example bitmaps of handwritten digits from U.S. postal envelopes. 

<img src="./handwritten_digits.png" width="400" align="center"/>

Each digit is represented by a $32 \times 32$ bitmap in which each element indicates one pixel with a value of white or black. Each $32 \times 32$ bitmap is divided into blocks of $4 \times 4$, and the number of white pixels are counted in each block. Therefore each handwritten digit is summarized by a vector $\mathbf{x} = (x_1, \ldots, x_{64})$ of length 64 where each element is a count between 0 and 16. 

We will use a model-based method by assuming a distribution on the count vector and carry out classification using probabilities. A common distribution for count vectors is the multinomial distribution. However as you will see in Q10, it is not a good model for handwritten digits. Let's work on a more flexible model for count vectors. In the Dirichlet-multinomial model, we assume the multinomial probabilities $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$, and density
$$
\begin{eqnarray*}
	\pi(\mathbf{p}) =  \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1},
\end{eqnarray*} 
$$
where $|\alpha|=\sum_{j=1}^d \alpha_j$.

## Q1

For a multivariate count vector $\mathbf{x}=(x_1,\ldots,x_d)$ with batch size $|\mathbf{x}|=\sum_{j=1}^d x_j$, show that the probability mass function for Dirichlet-multinomial distribution is
$$
    f(\mathbf{x} \mid \alpha) 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}  
    = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$
where $\Delta_d$ is the unit simplex in $d$ dimensions and $|\alpha| = \sum_{j=1}^d \alpha_j$.


### Answer to Q1  

According to the definition of probablity mass function, we can get the following:  
$$
f(\mathbf{x} \mid \alpha) 
    = \int_{\Delta_d}f(\mathbf{x} \mid \mathbf{p})\,f(\mathbf{p} \mid \alpha)\,d \mathbf{p} 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}   
$$
Then we will show it equals to the second part.  

$$
\begin{aligned}
 \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p} 
 = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
\end{aligned}
$$  

Since the density function of multinomial dirichlet distribution is $\pi(\mathbf{p})$, we can have:  

$$
\int\pi(\mathbf{p})\,d\mathbf{p} = \int \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1}\,d\mathbf{p} = 1
$$  

So 
$$  
\int\prod_{j=1}^d p_j^{\alpha_j-1} \,d\mathbf{p} = \frac{\prod_{j=1}^d \Gamma(\alpha_j)}{\Gamma(|\alpha|)}
$$  

Then 
$$
\begin{aligned}
f(\mathbf{x} \mid \alpha)
	&= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}\\
    &= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}}p_j^{x_j}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d  \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1}\, d \mathbf{p} \\
    &= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d  \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{x_j + \alpha_j-1} \, d \mathbf{p}
\end{aligned}
$$  

Plug in $\int\prod_{j=1}^d p_j^{\alpha_j-1} \,d\mathbf{p} = \frac{\prod_{j=1}^d \Gamma(\alpha_j)}{\Gamma(|\alpha|)}$  

$$
\begin{align*}
f(\mathbf{x} \mid \alpha) &=\binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(|\alpha|+|\mathbf{x}|)} \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \\ 
&= \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)} 
\end{align*}
$$

## Q2

Given independent data points $\mathbf{x}_1, \ldots, \mathbf{x}_n$, show that the log-likelihood is
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$
Is the log-likelihood a concave function?

### Answer to Q2 

From Q1, we can get the probablity mass function. Since $(x_{1}, x_{2}, ..., x_{i})$ are independent, we can take log of the probablity mass function for each $x_{i}$ and take the summation. Then we can get the log-likelihood function: 



$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$


This is not a concave function, and it can be proven by choosing a special case. We can choose $n = 1, x = (1, 0)$, and to prove $L(\alpha) = [\ln(\Gamma(\alpha_1 + 1) - \ln(\Gamma(\alpha_1)] - [\ln(\Gamma(\alpha_1+\alpha_2+1) - \ln(\Gamma(\alpha_1 + \alpha_2)]$ is not a concave function. Since $\Gamma(\alpha + 1) = (\alpha + 1)\Gamma(\alpha)$, 
then we need to prove $L(\alpha) = \ln(\alpha_1) - \ln(\alpha_1 + \alpha_2)$ is a nonconcave function, equivalently, we can prove $-L(\alpha) = -\ln(\alpha_1) + \ln(\alpha_1 + \alpha_2)$ is a nonconvex function. The Hessian matrix $-d^2L(\alpha)$ is:  
\begin{pmatrix}
\frac{1}{\alpha_1^2} - \frac{1}{(\alpha_1+\alpha_2)^2}  & - \frac{1}{(\alpha_1+\alpha_2)^2}\\ 
-\frac{1}{(\alpha_1+\alpha_2)^2} & - \frac{1}{(\alpha_1+\alpha_2)^2}
\end{pmatrix}

The matrix is clearly not PSD, so $-L(\alpha)$ is not convex, and $L(\alpha)$ is not concave. 

## Q3

Write Julia function to compute the log-density of the Dirichlet-multinomial distribution.

In [1]:
"""
    dirmult_logpdf(x::Vector, α::Vector)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::Vector, α::Vector)
    # your code here
    sum_x = sum(x)
    sum_α = sum(α)
    L = lgamma(sum_x + 1) - sum(lgamma.(x + 1)) + 
        sum(lgamma.(x + α)) - sum(lgamma.(α)) - 
        lgamma(sum_α + sum_x) + lgamma(sum_α)
    return L
end

function dirmult_logpdf!(r::Vector, X::Matrix, α::Vector)
    for j in 1:size(X, 2)
        r[j] = dirmult_logpdf(X[:, j], α)
    end
    return r
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at each data point in `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::Matrix, α::Vector)
    r = zeros(size(X, 2))
    dirmult_logpdf!(r, X, α)
end

dirmult_logpdf

In [2]:
using CSV

## Q4

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Each row contains the 64 counts of a digit and the last element (65th element) indicates what digit it is. For grading purpose, evaluate the total log-likelihood of this data at parameter values $\alpha=(1,\ldots,1)$ using your function in Q3.

In [3]:
# Read digits data
digits = CSV.read("optdigits.tra", datarow=1)
digits = Matrix{Float64}(digits);
X = digits[:, 1:64]';
Y = digits[:, 65];


In [4]:
α = ones(64)
r = dirmult_logpdf(X, α)
sum(r)

-638817.993292528

## Q5

Derive the score function $\nabla L(\alpha)$, observed information matrix $-d^2L(\alpha)$, and Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ for the Dirichlet-multinomial distribution.

Comment on why Fisher scoring method is inefficient for computing MLE in this example.

### Answer to Q5  

The derivative of $\ln(\Gamma(z))$ is defined as the following:  

$$
 \Psi(z) = \frac{d\ln\Gamma(z)}{dz} = \frac{\Gamma(z)^{'}}{\Gamma(z)}
$$

$$
 \Psi^{'}(z) = \frac{\Gamma(z)^{''}}{\Gamma(z)} - (\frac{\Gamma(z)^{'}}{\Gamma(z)})^{2}
$$  

Then we can derive the score function $\nabla L(\alpha)$:  

$$
\Theta_{j}=\frac{\partial L(\alpha)}{\partial \alpha_{j}} 
= \sum_{i=1}^n[\Psi(\alpha_{j} + x_{ij}) - \Psi(\alpha_{j})] - \sum_{i=1}^n[\Psi(|\alpha| + |x_{i}|) - \Psi(|\alpha|)]
$$

$$
\nabla L(\alpha) = \begin{pmatrix}
\Theta_1\\
\Theta_2\\
...\\
\Theta_d
\end{pmatrix}\\
$$

And the observed information matrix: 


$$
-\frac{\partial ^{2} L(\alpha)}{\partial \alpha_{j} \alpha_{k}} = \begin{cases}
	\sum_{i=1}^n[\Psi'(|\alpha| + |x_{j}|) - \Psi'(|\alpha|)] & \text{if $j \neq k$} \\
	-\sum_{i=1}^n[\Psi'(\alpha_{j} + x_{ij}) - \Psi'(\alpha_{j})] + 
    \sum_{i=1}^n[\Psi'(|\alpha| + |x_{i}|) - \Psi'(|\alpha|)] & \text{if $j == k$}
	\end{cases}
$$   


$$
-d^2L(\alpha) = \begin{pmatrix}
\Theta'_{11} & \Theta'_{12} & ... & \Theta'_{1d}\\ 
\Theta'_{21} & \Theta'_{22} & ... & \Theta'_{2d}\\ 
... & ... & ... & ...\\ 
\Theta'_{d1} & \Theta'_{d2} & ... & \Theta'_{dd}
\end{pmatrix}
$$  

After obtaining the observed information matrix, we can simply plug in and get the Fisher information matrix:  

$$
\mathbf{E}[-d^2L(\alpha)] = \mathbf{E}[\Theta'_{jk}] = \sum\Theta'_{jk}p(\Theta)  
$$

To use the Fisher information matrix, we need to compute the expected value of the observed matrix, which is really difficult, so the Fisher scoring method is inefficient for this problem

## Q6

What structure does the observed information matrix possess that can facilitate the evaluation of the Newton direction? Is the observed information matrix always positive definite? What remedy can we take if it fails to be positive definite? (Hint: HW1 Q6.)

### Answer to Q6  

The observed information matrix can be decomposed as following
$$
-d^2L(\alpha) = \mathbf{A} +
  \sum_{i=1}^n[\Psi'(|\alpha| + |x_{i}|) - \Psi'(|\alpha|)]\mathbf{1}\mathbf{1}^{T}\\
  \text{where $\mathbf{A}$ is a diagonal matrix } \\
  \mathbf{A} = -\sum_{i=1}^n[\Psi'(\alpha_{j} + x_{ij}) - \Psi'(\alpha_{j})] 
$$

To make use the special structure of the observed information matrix, we can use the Woodbury formula as showed in HW1 Q6. 

We can denote $\mathbf{A}$ as the diagonal matrix, $\mathbf{B} = \sum_{i=1}^n[\Psi'(|\alpha| + |x_{i}|) - \Psi'(|\alpha|)]$, $\mathbf{U} = \mathbf{1}$, and  $\mathbf{V} = \mathbf{1}$. Then the observed information matrix can be written as:  

$$
-d^2L(\alpha) = \mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T
$$


We can see the $\mathbf{B}$ is a scalar b, and we can use $c$ to represent $-b$. Then if we want the observed information matrix to be positive definite, the inverse of the matrix should also be PD.  

Then inverse of the oberseved information matrix can be represented as the following:


$$
	(\mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} + \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1}
$$  

And it can be written as:  

$$
	(\mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} + \mathbf{A}^{-1} \mathbf{U}
    \frac{c}{1 - c\mathbf{V}^T \mathbf{A}^{-1}\mathbf{U}} \mathbf{V}^T \mathbf{A}^{-1}
$$  

So the observed information matrix is PD, if $\frac{c}{1 - c\mathbf{V}^T \mathbf{A}^{-1}\mathbf{U}} > 0$. If it fails to be PD, we can use the identity matrix $\mathbf{I}$ to replace it. 


## Q7

Discuss how to choose a good starting point. Implement this as the default starting value in your function below. (Hint: Method of moment estimator may furnish a good starting point.)  

Moments of Dirichelet distribution:

$$
E[p_j] = \frac{\alpha_j}{\sum_{i}\alpha_i}
$$  

$$
Var[p_j] = \frac{\alpha_j(|\alpha|-\alpha_j)}{(1+|\alpha|)|\alpha|^{2}}
$$  

Since $E[p_j^{2}] = Var[p_j] + [E[p_j]]^{2}$, $E[p_k^{2}]$ can be written as the following: 

$$
\begin{align*}
E[p_j^{2}] &= \frac{\alpha_j(|\alpha|-\alpha_j)}{(1+|\alpha|)|\alpha|^{2}} + \frac{\alpha_j}{\sum_{i}\alpha_i}\\
&=\frac{\alpha_{j}|\alpha|(1+\alpha_j)}{(1+|\alpha|)|\alpha|^{2}} \\
&=E[p_j]\frac{1+\alpha_j}{1+|\alpha|}
\end{align*}
$$  

Then for a given dataset $(x_1, x_2, ..., x_i)$, we can compute the moment estimator:  

$$
\hat{E}[p_j] = \frac{\sum_{i=1}^n\frac{x_{ij}}{|x_i|}}{n} \\
\hat{E}[p_j^2] = \frac{\sum_{i=1}^n(\frac{x_{ij}}{|x_i|})^2}{n}
$$   

We can denote $v = \sum_{j=1}^d\frac{\hat{E}[p_j^2]}{\hat{E}[p_j]}$, and then we can have:  

$$
v = \frac{d+|\alpha|}{1+|\alpha|} \\
|\alpha| = \frac{d-1}{1-v} \\
$$

$$
\begin{align*}
\alpha_j &= \hat{E}[p_j]|\alpha| \\
&= \frac{\sum_{i=1}^n\frac{x_{ij}}{|x_i|}}{n}|\alpha| \\
&= \frac{\sum_{i=1}^n\frac{x_{ij}}{|x_i|}}{n} \frac{d-1}{1-v}
\end{align*}
$$


## Q8

Write a function for finding MLE of Dirichlet-multinomial distribution given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, using the Newton's method. 

In [5]:
"""
    dirmult_newton(X)

Find the MLE of Dirichlet-multinomial distribution using Newton's method.

# Argument
* `X`: an `n`-by-`d` matrix of counts; each column is one data point.

# Optional argument  
* `alpha0`: a `d` vector of starting point (optional). 
* `maxiters`: the maximum allowable Newton iterations (default 100). 
* `tolfun`: the tolerance for  relative change in objective values (default 1e-6). 

# Output
* `maximum`: the log-likelihood at MLE.   
* `estimate`: the MLE. 
* `gradient`: the gradient at MLE. 
* `hessian`: the Hessian at MLE. 
* `se`: a `d` vector of standard errors. 
* `iterations`: the number of iterations performed.
"""
# function dirmult_newton(X::Matrix; α0::Vector = nothing, 
            # maxiters::Int = 100, tolfun::Float64 = 1e-6)
function dirmult_newton(X::Matrix;
    maxiters::Int = 100, tolfun::Float64 = 1e-6)
    
    # set default starting point from Q7
    tot_n = size(X, 2)
    tot_d = size(X, 1)
    α1 = nothing
    α0 = nothing
    nonzero_indx = find(sum(X, 2) .!= 0)
    # rule zero zeros
    X = X[nonzero_indx, :]
    X_rowsum = sum(X, 1)
    n = size(X, 2)
    d = size(X, 1)
    
    # If the α is no initialized, use the moment estimator as a starting point
    if α0 == nothing
        p_hat = X ./ X_rowsum
        v = sum(sum(p_hat.^2, 2) ./ sum(p_hat, 2))
        α0 = sum(p_hat, 2) / n * (d - v) / (v - 1)
    end
    α0_sum = sum(α0)
    # calculate the log likelihood of this data at the starting point
    loglold = sum(dirmult_logpdf(X, squeeze(α0, 2)))
    dL = zeros(d, 1)
    ax = zeros(d, n)
    axa = zeros(d, n)
    inv_A = ones(d, 1)
    ax_abs = zeros(1, n)
    ax_tri = zeros(d, n)
    direct = zeros(d, 1)
    iterations = 0
    # Newton loop
    for iter in 1:maxiters
        # evaluate gradient (score)
        ax .= broadcast(+, X, α0)
        axa .= broadcast(-, digamma.(ax), digamma.(α0))
        ax_abs .= digamma.(broadcast(+, X_rowsum, α0_sum))
        # compute the score
        dL .= sum(axa, 2) - sum(ax_abs - digamma(α0_sum))
        # compute Newton's direction
        ax_tri = broadcast(-, trigamma.(ax), trigamma.(α0))
        inv_A .= -1 ./ sum(ax_tri, 2)

        # compute scaling factor c
        c = - sum(trigamma.(broadcast(+, α0_sum, X_rowsum)) - 
            trigamma(α0_sum))
        # the threshold to determin whether matrix is PD
        thres = sum(inv_A)
        if c < 1 / thres
            c = c
        else
            c = 0.99 * 1 / thres
        end
        # computer the direction
        direct .= inv_A .* dL + 
            (c / (1 - c * thres)) * inv_A * (inv_A' * dL)
        s_temp = - 0.99 * α0 ./ direct
        # ensure all steps are positive
        s_temp[find(s_temp .< 0)] = 10
        # get the minimum step
        s = minimum(s_temp)
        # line search loop
        # for lsiter in 1:10
        logl_new = loglold - 1
        while logl_new < loglold
            # step halving
            s = s / 2
            α1 = α0 + s * direct
            logl_new = sum(dirmult_logpdf(X, squeeze(α1, 2)))
        end
        # compute the loglikelihood
        logl = sum(dirmult_logpdf(X, squeeze(α1, 2)))
        # check convergence criterion
        if abs(logl - loglold) < tolfun * (abs(loglold) + 1)
            loglold = logl
            α0 = α1
            α0_sum = sum(α0)
            iterations = iterations + 1
            break;
        end
        loglold = logl
        α0 = α1
        α0_sum = sum(α0)
        iterations = iterations + 1
    end
    # compute logl, gradient, Hessian from final iterate
    c = - sum(trigamma.(broadcast(+, α0_sum, X_rowsum)) - 
            trigamma(α0_sum))
    H_tempt = - Diagonal(squeeze(sum(broadcast(-, 
        trigamma.(broadcast(+, X, α0)), 
        trigamma.(α0)), 2), 2)) - c * ones(d, d)
    hessian = zeros(tot_d, tot_d)
    hessian[nonzero_indx, nonzero_indx] = - H_tempt
    # output
    inv_A .= -1 ./ sum(broadcast(-, 
        trigamma.(broadcast(+, X, α0)), trigamma.(α0)), 2)
    thres = sum(inv_A)
    inv_H = Diagonal(Diagonal(squeeze(inv_A, 2)) + 
        c / (1 - c * thres) * (inv_A * inv_A'))[:]
    inv_H = inv_H[find(inv_H .!= 0)]
    # compute standard errors
    se = zeros(tot_d)
    se[nonzero_indx] = sqrt.(inv_H)
    dL = sum(broadcast(-, 
        digamma.(broadcast(+, X, α0)), digamma.(α0)), 2) - 
        sum(digamma.(broadcast(+, α0_sum, X_rowsum)) - 
        digamma(α0_sum))
    gradient = zeros(tot_d)
    fill!(gradient, sum(digamma.(broadcast(+,
        α0_sum, X_rowsum)) - digamma(α0_sum)))
    gradient[nonzero_indx] = dL
    estimate = zeros(tot_d)
    estimate[nonzero_indx] = α0
    maximum = loglold
    return maximum, estimate, gradient, 
        hessian, se, iterations
end

dirmult_newton

## Q9

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Find the MLE for the subset of digit 0, digit 1, ..., and digit 9 separately using your function. 

In [7]:
# Read digits data
digits = CSV.read("optdigits.tra", datarow=1)
digits = Matrix{Float64}(digits);
X = digits[:, 1:64]';
Y = digits[:, 65];

In [8]:
logl_dmul = zeros(10)
αs = zeros(size(X, 1), 10)
for i = 1:10
    mask_indx = find(Y .== (i - 1))
    cur_X = X[:, mask_indx]
    logl, α, _, _, _, _ = dirmult_newton(cur_X)
    αs[:, i] = α
    logl_dmul[i] = logl
end

In [9]:
logl_dmul

10-element Array{Float64,1}:
 -37358.4
 -42179.2
 -39985.3
 -40519.5
 -43488.8
 -41191.3
 -37702.5
 -40304.0
 -43130.8
 -43709.7

In [10]:
αs

64×10 Array{Float64,2}:
  0.0        0.0         0.0         …  0.0         0.0         0.0      
  0.037521   0.00799485  0.389224       0.139703    0.0880737   0.0762236
  5.00648    0.513408    3.82177        2.49721     2.49899     1.53114  
 14.9374     2.09512     5.25219        5.05992     5.83668     3.70754  
 12.1981     3.05279     2.51407        5.45268     5.58504     3.74031  
  2.46241    1.31915     0.311054    …  4.46356     2.42347     1.51328  
  0.0636524  0.13767     0.0157919      1.71971     0.199845    0.298747 
  0.0        0.0         0.0            0.17892     0.0         0.0159396
  0.0        0.0         0.0            0.0         0.00219805  0.0      
  1.02626    0.0572464   1.73584        0.254889    1.0232      0.667339 
 14.6382     1.03911     5.50771     …  3.17638     6.10708     4.02707  
 14.7992     3.23514     4.75994        3.99204     4.56584     3.35442  
 13.7833     4.09905     4.4493         4.17165     4.08871     3.29471  
  ⋮           

## Q10

As $\alpha / |\alpha| \to \mathbf{p}$, the Dirichlet-multinomial distribution converges to a multinomial with parameter $\mathbf{p}$. Therefore multinomial can be considered as a special Dirichlet-multinomial with $|\alpha|=\infty$. Perform a likelihood ratio test (LRT) whether Dirichlet-multinomial offers a better fit than multinomial for digits 0, 1, ..., 9 respectively.  

### Answer to Q10  

First, we can compute the MLE estimation with Dirichelet-multinomial from Q9. 

Then we can compute the loglikehood with the multinomial model:  

$$
\hat{P}_i = \frac{\alpha_i}{|\alpha|}
$$

In [11]:
function multinomial_est(X::Matrix)
    p = sum(X, 2) / sum(X)
    return p
end


function multinomial_mle(X::Matrix, p)
    p = p[find(p .!= 0)]
    X = X[find(sum(X, 2) .!= 0), :]
    X_rowsum = sum(X, 1)
    logl = sum(lgamma.(X_rowsum + 1) - 
        sum(lgamma.(X + 1), 1) + 
        sum(broadcast(*, X, log.(p)), 1))
    return logl
    
end

multinomial_mle (generic function with 1 method)

In [12]:
# Read digits data
digits = CSV.read("optdigits.tra", datarow=1)
digits = Matrix{Float64}(digits);
X = digits[:, 1:64]';
Y = digits[:, 65];
logl_mul = zeros(10)
for i = 1:10
    mask_indx = find(Y .== (i - 1))
    cur_X = X[:, mask_indx]
    p_mle = multinomial_est(cur_X)
    logl_mul[i] = multinomial_mle(cur_X, p_mle)
end

In [13]:
logl_mul

10-element Array{Float64,1}:
 -39592.2
 -54039.2
 -49111.5
 -47089.1
 -57344.1
 -51713.0
 -42597.3
 -49473.0
 -49695.9
 -54577.8

The LRT test begins with a comparison of the likelihood scores of the two models:  

$$
lR = 2 \times (\ln(x)_1 - \ln(x)_2)
$$  

And the LRT statistic approximately follows a Chi-square distribution.

In [14]:
LR = 2 * (logl_dmul - logl_mul)

10-element Array{Float64,1}:
  4467.56
 23720.0 
 18252.4 
 13139.2 
 27710.6 
 21043.4 
  9789.59
 18338.0 
 13130.0 
 21736.3 

In [15]:
# Pkg.add("RCall")
using RCall

In [16]:
P_tempt = R"""
pchisq($LR, 1)
"""

RCall.RObject{RCall.RealSxp}
 [1] 1 1 1 1 1 1 1 1 1 1


In [17]:
P_chi = 1 - collect(P_tempt)

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [18]:
using DataFrames

In [20]:
df = DataFrame(loglikelihood_Dirichlet = logl_dmul, 
    loglikelihood_Multi = logl_mul, 
    loglikelihood_Diff = LR, p_value = P_chi)

,loglikelihood_Dirichlet,loglikelihood_Multi,loglikelihood_Diff,p_value
1,-37358.4,-39592.2,4467.56,0.0
2,-42179.2,-54039.2,23720.0,0.0
3,-39985.3,-49111.5,18252.4,0.0
4,-40519.5,-47089.1,13139.2,0.0
5,-43488.8,-57344.1,27710.6,0.0
6,-41191.3,-51713.0,21043.4,0.0
7,-37702.5,-42597.3,9789.59,0.0
8,-40304.0,-49473.0,18338.0,0.0
9,-43130.8,-49695.9,13130.0,0.0
10,-43709.7,-54577.8,21736.3,0.0


We can see the loglikelihood of Dirichlet-multinomial distribution is significantly better than the loglikelihood of multinomial, so Dirichlet-multinomial offers a better fit than multinomial for digits 0, 1, ..., 9 respectively.

## Q11

Now we can construct a simple Bayesian rule for handwritten digits recognition:
$$
	\mathbf{x}	\mapsto \arg \max_k \widehat \pi_k f(x|\widehat \alpha_k).
$$
Here we can use the proportion of digit $k$ in the training set as the prior probability $\widehat \pi_k$. Report the performance of your classifier on the test set of 1797 digits in `optdigits.tes`.

In [21]:
# load training set
# Read digits data
digits = CSV.read("optdigits.tra", datarow=1)
digits = Matrix{Float64}(digits);
X = digits[:, 1:64]';
Y = digits[:, 65];
# compute the prior probablity
prior = zeros(10)
for i = 1:10
    prior[i] = sum(Y .== (i - 1))
end
prior = prior ./ sum(prior)

10-element Array{Float64,1}:
 0.0983521
 0.101753 
 0.0993984
 0.101753 
 0.101229 
 0.0983521
 0.0986137
 0.101229 
 0.0993984
 0.0999215

In [22]:
# read in test data
digits = CSV.read("optdigits.tes", datarow = 1)
digits = Matrix{Float64}(digits)
X_test = digits[:, 1:64]';
Y_test = digits[:, 65];

In [23]:
n_samples = size(X_test, 2)
logl_test = zeros(n_samples, 10)

for i = 1:10
    nonzero_indx = find(αs[:, i] .!= 0)
    cur_X_test = X_test[nonzero_indx, :]
    αs_tempt = αs[nonzero_indx, i]
    logl_test[:, i] = 
        prior[i] * dirmult_logpdf(cur_X_test, αs_tempt)
end

In [24]:
# Find maximum values and their corresponding indices
function findallmax(arr)
    max_positions = Vector{Int}()
    min_val = typemin(eltype(arr))
    for i in eachindex(arr)
        if arr[i] > min_val
            min_val = arr[i]
            empty!(max_positions)
            push!(max_positions, i)
        elseif arr[i] == min_val
            push!(max_positions, i)
        end
    end
    max_positions
end

findallmax (generic function with 1 method)

In [25]:
preds = zeros(n_samples)
for i = 1:n_samples
    preds[i] = findallmax(logl_test[i, :])[1] - 1
end

In [26]:
n_errs = sum(preds .!= Y_test)
err_rate = n_errs / n_samples

0.1229827490261547